In [3]:
from ghl.datasets.hetionet import HetionetDataset
import pandas as pd

OUT_FILE = "./gene_df.csv"
EMAIL = "ENTER EMAIL HERE"


ds = HetionetDataset("../tmp/hetionet/")
nodes_df = ds.get_nodes().query('node_id.str.startswith("Gene::")')
gene_ids = [s.replace("Gene::", "") for s in nodes_df["node_id"]]

from Bio import Entrez

Entrez.email = EMAIL


def get_gene_info(gene_ids):
    handle = Entrez.esummary(db="gene", id=",".join(gene_ids))
    result = Entrez.read(handle)
    handle.close()

    return result["DocumentSummarySet"]["DocumentSummary"]


from tqdm import tqdm

batch_size = 1000
gene_infos = []
for i in tqdm(range(0, len(gene_ids), batch_size)):
    gene_infos.extend(get_gene_info(gene_ids[i : i + batch_size]))

gene_names = [x["Name"] for x in gene_infos]
gene_df = pd.DataFrame(
    {
        "gene_name": gene_names,
        "gene_label": gene_ids,
        "gene_id": nodes_df.index,
    }
)
gene_df.to_csv(OUT_FILE, index=False)

100%|██████████| 21/21 [00:57<00:00,  2.72s/it]


In [4]:
gene_df

,gene_name,gene_label,gene_id
0,A1BG,1,14863
1,NAT2,10,14864
2,ADA,100,14865
3,CDH2,1000,14866
4,AKT3,10000,14867
...,...,...,...
20940,PTBP3,9991,35803
20941,KCNE2,9992,35804
20942,DGCR2,9993,35805
20943,CASP8AP2,9994,35806
